[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/juansensio/blog/blob/master/118_falcon/118_falcon.ipynb)

# Falcon 🦅

Es innegable que estamos viviendo en tiempos interesantes para el desarrollo de la Inteligencia Artificial (IA), especialmente en el campo del procesamiento de lenguaje natural (NLP). Y es que parece que cada día aparece un nuevo modelo de lenguaje (LLM, *Large Languaje Model*) mejor que el anterior. En este post vamos a hablar sobre el último de estos modelos (por lo que si estás leyendo este post en unas semanas después de su publicación puede que ya haya quedado obsoleto). Este modelo tiene el nombre de [Falcon](https://falconllm.tii.ae/) y tiene unas características muy interesantes.

Los responsables de Falcon son el *Technology Innovation Institute* ([TII](https://www.tii.ae/)), un centro de investigación de Abu Dhabi.

Se han presentado dos modelos de diferente tamaño: [Falcon 7B](https://huggingface.co/tiiuae/falcon-7b) y [Falcon 40B](https://huggingface.co/tiiuae/falcon-40b), el primero con 7.000 millones de parámetros y el segundo con 40.000 millones. Para que te hagas una idea, GPT-3 tiene 175.000 millones de parámetros, mientras que LLaMA ofrece versiones entre los 7.000 y 65.000 millones. La versión mayor del modelo sostenta actualmente las primeras posiviones en la [Open LLM Leaderdoard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard) de Huggingface.

![leaderboard](lb.png)


En este *ranking* se evalúan los diferentes modelos en 4 *benchmarks* populares diseñados para evluar las capacidades a nivel de conocimiento y razonamiento en diversos campos. ¿Como es posible entonces que este modelo sea mejor siendo más "pequeño"? Pues el secreto está en su [Dataset](https://falconllm.tii.ae/Falcon_LLM_RefinedWeb.pdf).  

Los grandes modelos como GPT-3 o PaLM dan más importancia a la cantidad de parámetros que a la cantidad de *tokens* con los que son entrenados. Sin embargo, DeepMind demostró que era posible obtener resultados similares, o incluso mejores, con modelos más pequeños pero entrenados durante más tiempo, usando más *tokens*. De este estudio nacieron modelos como [Chinchilla](https://arxiv.org/abs/2203.15556) o los más actuales [LLaMA](https://arxiv.org/abs/2302.13971). En todos los casos, el texto usado para entrenar estos modelos es texto público de la web, combinado con código de Github, libros, artículos científicos y otras fuentes diversas. El problema es que si queremos modelos más capaces (con más parámetros), necesitamos más datos. Y el contenido de calidad (libros, artículos científicos, ...) es, por un lado, limitado y, por otro, no es fácil de obtener y procesar. Es en este punto en el que entra en juego el dataset RefinedWeb, usado para entrenar Falcon. Este dataset consiste únicamente en texto extraído de internet, que combinado con las técnicas de curado adecuadas, permite generar un dataset de 5 trillones de *tokens*. Falcon fue entrenado en 1 *trillon*, durante 2 meses, usando 384 GPUs. En comparación, modelos como GPT-3 o PaLM no superan el trillón (y las estimaciones de los nuevos modelos GPT-4 o PaLM-2 hablan de 2-3 trillones). Por otro lado, LLaMA usa 1.4 trillones de *tokens*, demostrando que el uso de datos extraídos únicamente de internet pero con el procesado adecaudo puede ser una alternativa viable a los datos de calidad, con la increíble reducción de costes que esto supone lo cual además hace muchísimo más accesible el entrenamiento de LLMs a nivel de estado del arte en diferentes idiomas y dominios.

![dataset](dataset.png)

A falta de la publicación oficial, las principales caracterísitcas de Falcon son el uso de [*Flash Attention*](https://github.com/HazyResearch/flash-attention) y [Multiquery](https://arxiv.org/abs/1911.02150), dos técnicas diseñadas para mejorar la eficiencia de los modelos de lenguaje durante el entrenamiento y, sobre todo, en inferencia.

![flash](https://github.com/HazyResearch/flash-attention/raw/main/assets/flashattn_banner.jpg)


Otro aspecto muy interesante es que estos modelos tienen una licencia [Apache 2.0](https://www.apache.org/licenses/LICENSE-2.0) muy permisiva y que permite su uso para aplicaciones comerciales sin ningún tipo de restricción o *royalties*. Esto tiene implicaciones muy importantes, ya que significa que cualquier empresa puede utilizar estos modelos para sus propias aplicaciones, ya sea aplicaciones internas que requieran de NLP o aplicaciones comerciales que se vendan a terceros. En el caso del uso interno tiene además otra ventaja muy clara para todas aquellas aplicaciones que no quieran tener que copmpartir sus datos con sistemas externos, como la API de OpenAI por ejemplo. Esto es de especial importancia en regiones como Europa, donde la legislación de protección de datos es muy estricta.

## Usando Falcon

In [1]:
from transformers import AutoTokenizer
import transformers
import torch

model = "tiiuae/falcon-7b"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    "¿Quién es Juan Sensio?",
    max_length=200,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerNormForCausalLM', 'RoCBertForCau

Result: ¿Quién es Juan Sensio?
¿Quién es Juan Sensio? (Who is John Sensio) is a 1997 American romantic comedy movie starring John Travolta, Tim Daly, William Devane, and Kelly Preston.
The movie is an adaptation of the 1992 Italian movie La solitudine dei numeri primi by Dino Risi.
The movie was produced by John Travolta through his production company, New Line Cinema, and released on October 9, 1997 through Paramount Pictures.
¿Quién es John Sensio (Trailer)
¿Quién es John Sensio (trailer)
¿Quién es Juan Sensio? is available now:
- In theaters for 1 hour, 45 minutes
- On DVD for 1 hour, 46 minutes
- To buy the DVD (US)
- To buy the DVD (UK)
- To buy the


In [2]:
import langchain

langchain.__version__

'0.0.189'

In [14]:
from langchain import HuggingFacePipeline

# OJO! max_length tiene que ser suficiente como para tener el documento (chuck) + el prompt + el system prompt + respuesta generada !!!
llm = HuggingFacePipeline.from_model_id(
    model_id="tiiuae/falcon-7b-instruct", 
    task="text-generation", 
    model_kwargs={
        "max_length": 1024, 
        'do_sample': True,
        'top_k': 10,
        'num_return_sequences': 1,
        'device_map': 'auto', 
        'trust_remote_code': True,
        'torch_dtype': torch.bfloat16
    },
)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device has 2 GPUs available. Provide device={deviceId} to `from_model_id` to use availableGPUs for execution. deviceId is -1 (default) for CPU and can be a positive integer associated with CUDA device id.
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'Peg

In [15]:
from langchain.document_loaders import OnlinePDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain

loader = OnlinePDFLoader("https://arxiv.org/pdf/1911.01547.pdf")
document = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
documents = text_splitter.split_documents(document)
embeddings = HuggingFaceEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

qa = ConversationalRetrievalChain.from_llm(
    llm, 
    vectorstore.as_retriever(), 
    return_source_documents=True
)

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
Created a chunk of size 1995, which is longer than the specified 1024
Created a chunk of size 1435, which is longer than the specified 1024
Created a chunk of size 1167, which is longer than the specified 1024
Created a chunk of size 1342, which is longer than the specified 1024
Created a chunk of size 1037, which is longer than the specified 1024
Created a chunk of size 1484, which is longer than the specified 1024
Created a chunk of size 1057, which is longer than the specified 1024
Created a chunk of size 1044, which is longer than the specified 1024
Created a chunk of size 1120, which is longer than the specified 1024
Created a chunk of size 1268, which is longer than the specified 1024
Created a chunk of size 1464, which is longer than the specified 1024
Created a chunk of size 1898, which is longer than the specified 1024
Created a chunk of size 1066, whi

In [16]:
chat_history = []
query = "What is the definition of intelligence?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
/home/juan/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (1024) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_2275569/2066281337.py:3 in <module>                                               │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2275569/2066281337.py'                      │
│                                                                                                  │
│ /home/juan/.local/lib/python3.10/site-packages/langchain/chains/base.py:140 in __call__          │
│                                                                                                  │
│   137 │   │   │   )                                                                              │
│   138 │   │   except (KeyboardInterrupt, Exception) as e:                                        │
│   139 │   │   │   run_manager.on_chain_error(e)                                                  │
│ ❱ 140 │   │   │   raise e                                                                        │
│   141 │   │   run_manager.on_chain_end(outputs)                                                  │
│   142 │   │   return self.prep_outputs(inputs, outputs, return_only_outputs)                     │
│   143                                                                                            │
│                                                                                                  │
│ /home/juan/.local/lib/python3.10/site-packages/langchain/chains/base.py:134 in __call__          │
│                                                                                                  │
│   131 │   │   )                                                                                  │
│   132 │   │   try:                                                                               │
│   133 │   │   │   outputs = (                                                                    │
│ ❱ 134 │   │   │   │   self._call(inputs, run_manager=run_manager)                                │
│   135 │   │   │   │   if new_arg_supported                                                       │
│   136 │   │   │   │   else self._call(inputs)                                                    │
│   137 │   │   │   )                                                                              │
│                                                                                                  │
│ /home/juan/.local/lib/python3.10/site-packages/langchain/chains/conversational_retrieval/base.py │
│ :110 in _call                                                                                    │
│                                                                                                  │
│   107 │   │   new_inputs = inputs.copy()                                                         │
│   108 │   │   new_inputs["question"] = new_question                                              │
│   109 │   │   new_inputs["chat_history"] = chat_history_str                                      │
│ ❱ 110 │   │   answer = self.combine_docs_chain.run(                                              │
│   111 │   │   │   input_documents=docs, callbacks=_run_manager.get_child(), **new_inputs         │
│   112 │   │   )                                                                                  │
│   113 │   │   if self.return_source_documents:                                                   │
│                                                                                                  │
│ /home/juan/.local/lib/python3.10/site-packages/langchain/chains/base.py:239 in run               │
│                                                                                                  │
│   236 │   │   │   return self(args[0], callbacks=callbacks)[self.output_keys[0]]                 │
│   237 │   │                                                

In [ ]:
chat_history = [(query, result["answer"])]
# chat_history = []
query = "Who is the author of the paper?"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Input length of input_ids is 998, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


' David'

In [ ]:
for doc in result['source_documents']:
    print(doc.page_content)

It clearly spells out that the process of creating an intelligent system can be ap- proached as an optimization problem, where the objective function would be a com- putable approximation of our quantitative intelligence formula. As pointed out in II.2.2, this objective function could be further reﬁned by incorporating regularization terms that would take into account alternative forms of efﬁciency.

It encourages a focus on developing broad or general-purpose abilities rather than pursuing skill alone, by proposing a target metric that penalises excessive reliance on experience or priors, and discounting tasks that feature low generalization difﬁculty.
Charles Darwin, 1859

The evolutionary psychology view of human nature is that much of the human cognitive function is the result of special-purpose adaptations that arose to solve speciﬁc problems encountered by humans throughout their evolution (see e.g. [19, 74]) – an idea which orig- inated with Darwin [21] and that coalesced in the